# Creación de un modelo de Machine Learning para planes de telefonía

# Contenido

* [Introducción](#)
* [Objetivos](#)
* [Etapas](#)
* [Inicialización: descripción de datos](#)
    * [Cargar datos](#)
* [Segmentación de datos](#)
* [Prueba y entrenamiento de modelos de Machine Learning](#)
* [Prueba de calidad del modelo](#)
* [Prueba de cordura](#)
* [Conclusiones](#)


# Introducción

La compañía de telecomunicaciones móviles Megaline, viendo el análisis realizado previamente en el estudio de sus planes de telefonía, ahora nos propone un nuevo reto y análisis al ver que muchos de sus clientes utilizan planes heredados y que no siempre se ajustan a lo que ellos necesitan. Megaline quiere que desarrollemos un modelo de Machine Learning que pueda analizar el comportamiento de sus clientes y (pronosticar) recomendar uno de los planes de la empresa: Smart o Ultra. Para el desarrollo de este proyecto utilizaremos modelos de aprendizaje automático supervisados, en este caso, los de clasificación.  

# Objetivos

*Debido a que nuestro objetivo es el modelamiento, no tenemos hipótesis estadísticas a probar en específico esta vez.* 

Nuestro principal objetivo será determinar el modelo con la mayor exactitud posible, teniendo como umbral de exactitud un 0.75.

Como tarea adicional, probaremos la calidad del modelo y realizaremos una prueba de cordura.

# Etapas del análisis

En este proyecto, seguiremos un esquema necesario para solucionar el problema de la creación y prueba de un modelo de Machine Learning. Para esto, seguiremos a grandes rasgos el siguiente proceso:

1. Descripción de los datos
2. Segmentación de los datos
3. Prueba y entrenamiento de los modelos de Machine Learning
4. Prueba de calidad del modelo
5. Prueba de cordura
6. Conclusiones finales

Debido a que el preprocesamiento y análisis exploratorio fueron realizados en el anterior informe, en este presente trabajo se obviarán esos pasos y nos enfocaremos en el modelamiento luego de describir nuestros datos.

# Inicialización: descripción de los datos

**Carga de librerías necesarias**

In [2]:
import numpy as np 
import pandas as pd # librerías estándar para manipular datos

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression  # librerías de sklearn para los modelos y métricas

## Cargar los datos

Tenemos solo un dataset con los datos finales y preprocesados:

In [3]:
try:
    data_megaline = pd.read_csv("users_behavior.csv")
except:
    data_megaline = pd.read_csv("datasets/users_behavior.csv")

# Exploración inicial de datos

Nuestra tabla contiene la siguiente información según su documentación:

* ``calls``: número de llamadas.
* ``minutes``: duración total de la llamada en minutos.
* ``messages``: número de mensajes de texto.
* ``mb_used``: tráfico de internet utilizado en MB.
* ``is_ultra``: si el plan utilizado es el *Ultra* (1), de lo contrario, es Smart (0)

Ahora podemos explorar la tabla:

In [4]:
data_megaline.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
data_megaline.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [6]:
data_megaline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


De lo mostrado previamente, podemos notar que ninguna de nuestra variables cuenta con problemas grandes en su distribución, tipos de datos y valores ausentes. El motivo es precisamente el hecho de que el preprocesamiento ya fue realizado en el anterior trabajo. 

Ahora que conocemos a nuestras variables y sabemos que estas no presentan problemas para su uso, nos enfocaremos en nuestra posible variable objetivo: ``is_ultra``:

In [8]:
data_megaline['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

El 30 % de los usuarios tienen registrado el plan *Ultra* en su suscripción a la empresa Megaline, mientras que el 70 % restante pertenece al plan *Smart*. Esta diferencia muestra y sugiere la existencia de un ligero desbalance de nuestra variable objetivo. Debido a que el problema no se ve lo suficientemente grave como para preocuparnos por él, no se abordará el manejo de este problema por ahora y esa será la limitación de este proyecto.

## Conclusiones del apartado

En este apartado observamos las principales características y dimos un vistazo general a nuestras variables. No notamos problemas que requieran una etapa de procesado extra debido a que este proceso fue realizado en el trabajo previo. También notamos un ligero desbalance en nuestra variable objetivo, pero se concluyó que su proporción no es grave ni posiblemente genere problemas en la estimación y evaluación de nuestro modelo.

# Segmentación de los datos

Debido a que realizaremos un modelo de clasificación predictivo, tendremos que definir una especie de relación de nuestra variable objetivo (target) a pronosticar y sus características (features). Ya habíamos anticipado este problema designando a ``is_ultra`` como nuestra variable objetivo, pero continuaremos buscando la relación en un modelo teórico no formal: 

Nuestra variable objetivo (``is_ultra``) teóricamente se relaciona con las variables ``calls``, ``minutes``, ``messages``, y ``mb_used`` (características). Es decir, existe una relación teórica no formal en la que esas características determinarán si el cliente usa un respectivo plan o no (variable objetivo - ``is_ultra``). En la vida real, esta relación tiene sentido y esa será la relación que nos ayude a generar nuestro modelo predictivo.

Guardaremos nuestra semilla generadora de números pseudoaleatorios en la variable ``random_state``.

A continuación, separaremos nuestro dataset en ``target`` y ``features``.

Luego de lo anterior, necesitamos dividir nuestros datos para las respectivas etapas de entrenamiento y validación de nuestro modelo. Usaremos la función "train_test_split" para este objetivo, y emplearemos la proporción 3:1 al dividir los datos (75 % para entrenamiento, 25 % para validación):

In [9]:
random_state = 200

In [10]:
target = data_megaline['is_ultra']
features = data_megaline.drop('is_ultra', axis=1)

In [11]:
target_train, target_valid, features_train, features_valid = train_test_split(target, features, test_size=0.25, random_state=random_state)

Verificaremos obteniendo el atributo *shape* de cada dataset:

In [12]:
target_train.shape

(2410,)

In [13]:
target_valid.shape

(804,)

In [14]:
features_train.shape

(2410, 4)

In [15]:
features_valid.shape

(804, 4)

## Pequeñas conclusiones del apartado

En este apartado definimos la relación que nos permitirá la estimación del modelo, siendo ``is_ultra`` nuestra variable objetivo y las demás variables las características del modelo.

Luego de definir estas variables, las separamos y, a continuación, dividimos nuestro dataset para el entrenamiento (75 %) y validación (25 %). Finalmente, nos aseguramos de que la división se haya realizado correctamente.

# Prueba y entrenamiento de los modelos de Machine Learning